In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import tigre

import numpy as np
import matplotlib.pyplot as plt

from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

# Load the mat files and extract the data
import scipy.io
import os

In [3]:


# Specify the directory where the mat files are located
directory = '/media/jericho/T7/Jericho_MAR_Studies/'

# Get the list of all the mat files in the directory
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mat')]
files.sort()

# Load the mat files into a pandas dataframe
data = []
for file in files:
    data.append(scipy.io.loadmat(file))


In [4]:

# Extract the data from the mat files into a pandas dataframe
# dict_keys(['__header__', '__version__', '__globals__', 'L', 'metadata'])
data_all = {}
for i in range(len(data)):
    f_name = files[i].split('/')[-1]
    data_all[f_name] = {}
    data_all[f_name]['projections'] = data[i]['L']
    data_all[f_name]['metadata'] = data[i]['metadata']


In [5]:
# first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
# first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
# secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
# secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [6]:
first_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Spine NoMetal kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Spine NoMetal kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [67]:
first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_SingleScrew kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_SingleScrew kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [68]:
def inpaint_images_spine(images0,images1):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    
    for image_high, image_low in zip(images2, images3):

        ratio0 = image_high/(image_low)

        mask = np.zeros_like(image_high)
        mask[x_ind:,125:145 ] = ratio0[x_ind:,125:145 ] > 1.05
        image_high[mask.astype(bool)] = np.NaN
        expanded_mask = binary_dilation(mask)

        expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        results.append(image_result)
        # results.append(image_high)
    return results

images_LI_second = inpaint_images_spine(secon_layer_bilat_proj.T[:],first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_20552/4158768919.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_20552/4158768919.py:8: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)


In [10]:
plt.figure()
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
m1 = first_layer_bilat_proj.T > 6.8
ratio[first_layer_bilat_proj.T < 5] = 0
m0 = ratio > 1.01
m2 = ratio > 1.03
m3 = ratio > 1.05
m4 = ratio > 1.07

m5 = m1.astype(int) + m2 + m3 + m4 + m0
m6 = m5 > 1

m6 = ratio/(first_layer_bilat_proj.T)
m7 = m6 < 0.15
# m7 = m6 > np.nanpercentile(m6, 99)
# aval = first_layer_bilat_proj - 
m8 = m1.astype(int) + m7 + m2
m8[first_layer_bilat_proj.T < 5] = 0
pli.hyperslicer(m8)

/tmp/ipykernel_20552/490857703.py:2: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/490857703.py:2: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/490857703.py:13: RuntimeWarning: invalid value encountered in divide
  m6 = ratio/(first_layer_bilat_proj.T)


AttributeError: 'Controls' object has no attribute 'vbox'

In [70]:
plt.figure(figsize=(5,10))
plt.subplot(211)
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
plt.imshow(first_layer_bilat_proj.T[363],cmap='gray')
plt.title('Projection image')
plt.axvline(x=131, color='indianred', linestyle=':')
plt.axvline(x=139, color='indianred', linestyle=':')
ratio[np.isinf(ratio)] = 0
ratio[first_layer_bilat_proj.T<6] = 0
ratio[ratio > 1.07] = 0
ratio[ratio < 1.01] = 0
plt.axis('off')
plt.subplot(212)
plt.plot(np.nanmean(ratio, axis=(0,1)))
plt.xlabel('Pixel (Axial direction)')
plt.ylabel('Mean ratio $\mu_{high}/\mu_{low}$')
plt.title('Mean ratio $\mu_{high}/\mu_{low}$')
# draw a red line horizontally at y = 0.7
plt.axhline(y=0.07, color='darkorange', linestyle='--',label='Threshold')
# draw horizontal line at x=131 and x=139
plt.axvline(x=131, color='indianred', linestyle=':')
plt.axvline(x=139, color='indianred', linestyle=':',label='ROI')
plt.legend()
plt.tight_layout()


/tmp/ipykernel_20552/657936292.py:3: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/657936292.py:3: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]


In [71]:
from scipy.ndimage import gaussian_filter
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
plt.figure(figsize=(5,10))
plt.subplot(211)
plt.imshow(first_layer_bilat_proj.T[138],cmap='gray')
plt.title('Projection image')
plt.axvline(x=131, color='indianred', linestyle=':')
plt.axvline(x=139, color='indianred', linestyle=':')
ratio[np.isinf(ratio)] = 0
ratio[first_layer_bilat_proj.T<4] = 0
ratio[ratio > 1.07] = 0
ratio[ratio < 0.8] = 0
plt.axis('off')
plt.subplot(212)
val_adj = np.nanmean(ratio[[200,400,600],x_ind:,111:129], axis=(2)).T
val_roi = np.nanmean(ratio[[200,400,600],x_ind:,131:139], axis=(2)).T
ratio_roi = val_roi/val_adj
colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
# gaussian smooth the ratio
for i, ratio in enumerate(ratio_roi.T):
    ratio_roi[:,i] = gaussian_filter(ratio, sigma=2)
# ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
for i, ratio in enumerate(ratio_roi.T):
    plt.plot(ratio,'--', color=colors[i])
plt.xlabel('Pixel (Axial direction)')
plt.ylabel('Mean ratio $\mu_{high}/\mu_{low}$')
plt.title('Mean ratio $\mu_{high}/\mu_{low}$')
# Add an axis on the right side of the plot
plt.twinx()
val_adj = np.nanmean(first_layer_bilat_proj.T[[200,400,600],x_ind:,111:129], axis=(2)).T
val_roi = np.nanmean(first_layer_bilat_proj.T[[200,400,600],x_ind:,131:139], axis=(2)).T
ratio_roi = val_roi/val_adj
for i, ratio in enumerate(ratio_roi.T):
    ratio_roi[:,i] = gaussian_filter(ratio, sigma=2)
for i, ratio in enumerate(ratio_roi.T):
    plt.plot(ratio,':', color=colors[i])

# draw a red line horizontally at y = 0.7
# plt.axhline(y=0.07, color='darkorange', linestyle='--',label='Threshold')
# draw horizontal line at x=131 and x=139
# plt.axvline(x=131, color='indianred', linestyle=':')
# plt.axvline(x=139, color='indianred', linestyle=':',label='ROI')
plt.legend()
plt.tight_layout()

/tmp/ipykernel_20552/4117994670.py:2: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/4117994670.py:2: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [72]:
from scipy.ndimage import gaussian_filter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

x_ind = 240
slice_ind = 253
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
plt.figure(figsize=(5,10))
plt.subplot(211)
plt.imshow(first_layer_bilat_proj.T[363].T,cmap='gray')
# gt = 2 > (first_layer_bilat_proj.T[slice_ind].T/first_layer_bilat_proj_nm.T[slice_ind].T).astype(np.float32) > 1.05
ratio2 = first_layer_bilat_proj.T[slice_ind]/(first_layer_bilat_proj_nm.T[slice_ind])
ratio2[first_layer_bilat_proj.T[slice_ind] < 5] = 0
# plt.axis('off')
gt = (ratio2 > 1.05).astype(np.float32)
plt.imshow(first_layer_bilat_proj.T[slice_ind].T,cmap='gray',alpha=1)
gt[gt==0] = np.NaN
plt.imshow(gt.T,cmap='flag',alpha=0.2,label='Ground truth')
plt.title('Projection image')
plt.axhline(y=131, color='cornflowerblue', linestyle=':')
plt.axhline(y=139, color='cornflowerblue', linestyle=':',label='ROI')
plt.axvline(x=307, color='forestgreen', linestyle=':')
plt.axvline(x=256, color='forestgreen', linestyle=':')
plt.axvline(x=332, color='forestgreen', linestyle=':',label='Metal Boundaries')
plt.plot(1,1,'r-',label='Ground Truth')
ratio[np.isinf(ratio)] = 0
ratio[first_layer_bilat_proj.T<4] = 0
ratio[ratio > 1.07] = 0
ratio[ratio < 0.8] = 0
plt.axis('off')
plt.legend(loc=3)
# Make a zoom inset axis for the ROI region
# Inset axes
ax = plt.gca()


axins = zoomed_inset_axes(ax, zoom=2, loc=2)
axins.imshow(first_layer_bilat_proj.T[slice_ind].T,cmap='gray')
plt.axis('off')
# Specify the limits
x1, x2, y1, y2 = x_ind, 350,120, 149 # specify the limits
axins.set_xlim(x1, x2)  # apply the x-limits
axins.set_ylim(y1, y2)  # apply the y-limits

# Indicate zoom location

mark_inset(ax, axins, loc1=1, loc2=3, fc="none", ec="0.5")

plt.subplot(212)
val_adj = np.nanmean(ratio[slice_ind,x_ind:,111:129], axis=(1)).T
val_roi = np.nanmean(ratio[slice_ind,x_ind:,131:139], axis=(1)).T
ratio_roi = val_roi/val_adj
colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
# gaussian smooth the ratio
# for i, ratio in enumerate(ratio_roi.T):
ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
# for i, ratio in enumerate(ratio_roi.T):
plt.plot(ratio_roi.T,'--', color='darkorange',label='$\mu_{high}/\mu_{low}$')
plt.axhline(y=1.03, color='darkorange', linestyle='-',label='Threshold')
plt.xlabel('Pixel')
plt.ylabel('$\mu_{high}/\mu_{low}$')
plt.title('Metal Selection')
plt.legend()
# Add an axis on the right side of the plot
plt.twinx()
val_adj = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,105:129], axis=(1)).T
val_roi = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,131:139], axis=(1)).T
ratio_roi = val_roi/val_adj

ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# for i, ratio in enumerate(ratio_roi.T):
plt.plot(ratio_roi,'-.', color='indianred',label='$\mu_{high}$')
plt.ylabel('Relative $\mu_{high}$')
# plot the groundtruth as a red line
# gt_line = np.nansum(gt.T[129:142,x_ind:].astype(np.float32),axis=0)
# gt_line[gt_line==0] = np.NaN
# gt_line[gt_line > 0] = 1
# plt.plot(gt_line,'b-',label='Ground Truth')
# val_adj = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,111:129], axis=(1)).T
# val_roi = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,131:139], axis=(1)).T
# ratio_roi = val_roi/val_adj

# ratio_roi = gaussian_filter(ratio_roi, sigma=2)

# plt.plot(ratio,':', color=colors[i])

# draw a red line horizontally at y = 0.7
# plt.axhline(y=0.07, color='darkorange', linestyle='--',label='Threshold')
# draw horizontal line at x=131 and x=139
# plt.axvline(x=18, color='forestgreen', linestyle=':')
# plt.axvline(x=68, color='forestgreen', linestyle=':',label='Metal Boundaries')
# plt.axvline(x=93, color='indianred', linestyle=':')
# Convert the axvlines to regions of forestgreen between them
plt.fill_betweenx([0.98,1.4], 18, 68, color='forestgreen', alpha=0.2,label='Metal Region')
plt.fill_betweenx([0.98,1.4], 93, 112, color='forestgreen', alpha=0.2)
plt.axhline(y=1.1, color='indianred', linestyle='-',label='Threshold')
plt.legend()
plt.tight_layout()

/tmp/ipykernel_20552/1426257323.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/1426257323.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/1426257323.py:101: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [73]:
from scipy.ndimage import gaussian_filter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

x_ind = 240
slice_ind = 253
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
plt.figure(figsize=(8,8))
plt.subplot(222)
# plt.imshow(first_layer_bilat_proj.T[slice_ind].T,cmap='gray')
# gt = 2 > (first_layer_bilat_proj.T[slice_ind].T/first_layer_bilat_proj_nm.T[slice_ind].T).astype(np.float32) > 1.05
ratio2 = first_layer_bilat_proj.T[slice_ind]/(first_layer_bilat_proj_nm.T[slice_ind])
ratio2[first_layer_bilat_proj.T[slice_ind] < 5] = 0
# plt.axis('off')
gt = (ratio2 > 1.05).astype(np.float32)
plt.imshow(first_layer_bilat_proj.T[slice_ind].T[50:,150:],cmap='gray',alpha=1)
gt[gt==0] = np.NaN
plt.imshow(gt.T[50:,150:],cmap='flag',alpha=0.2,label='Ground truth')
plt.title('(b) Projection image')
plt.axhline(y=131-50, color='cornflowerblue', linestyle=':')
plt.axhline(y=139-50, color='cornflowerblue', linestyle=':',label='Vertical ROI')
plt.axvline(x=307-150, color='indianred', linestyle=':')
plt.axvline(x=256-150, color='indianred', linestyle=':')
plt.axvline(x=332-150, color='indianred', linestyle=':',label='Horizontal ROI')
# Turn these into a fill between

# plt.plot(1,1,'r-',label='Ground Truth')
ratio[np.isinf(ratio)] = 0
ratio[first_layer_bilat_proj.T<4] = 0
ratio[ratio > 1.07] = 0
ratio[ratio < 0.8] = 0
plt.axis('off')
plt.legend(loc=3)
# Make a zoom inset axis for the ROI region
# Inset axes
ax = plt.gca()


axins = zoomed_inset_axes(ax, zoom=1.5, loc=2)
axins.imshow(first_layer_bilat_proj.T[slice_ind].T[50:,150:],cmap='gray')
plt.fill_betweenx([0, first_layer_bilat_proj.T[slice_ind].T[50:,150:].shape[0]-2], 256-150, 307-150, color='indianred', alpha=0.2)#,label='Metal Region')
plt.fill_betweenx([0, first_layer_bilat_proj.T[slice_ind].T[50:,150:].shape[0]-2], 332-150, 355-150, color='indianred', alpha=0.2)
plt.yticks([])
plt.xticks([90,140,190],labels=[0,50,100])
# Specify the limits
x1, x2, y1, y2 = x_ind-150, 350-150,120-50, 149 -50# specify the limits
axins.set_xlim(x1, x2)  # apply the x-limits
axins.set_ylim(y1, y2)  # apply the y-limits

# Indicate zoom location

mark_inset(ax, axins, loc1=3, loc2=1, fc="none", ec="0.5")

plt.subplot(224)
val_adj = np.nanmean(ratio[slice_ind,x_ind:,111:129], axis=(1)).T
val_roi = np.nanmean(ratio[slice_ind,x_ind:,131:139], axis=(1)).T
ratio_roi = val_roi/val_adj
colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
# gaussian smooth the ratio
# for i, ratio in enumerate(ratio_roi.T):
ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# colors = ['darkorange', 'indianred', 'forestgreen', 'steelblue']
# for i, ratio in enumerate(ratio_roi.T):
plt.plot(ratio_roi.T,'-', color=colors[0],label='$\mu_{high}/\mu_{low}$')
plt.axhline(y=1.03, color='gray', linestyle='-.',label='Threshold 2')
plt.xlabel('Pixel (horizontal direction)')
plt.ylabel('$\mu_{high}/\mu_{low}$')
plt.title('(d) Horizontal Metal Selection')
plt.legend(loc='upper right')
# Add an axis on the right side of the plot
plt.twinx()
val_adj = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,105:129], axis=(1)).T
val_roi = np.nanmean(first_layer_bilat_proj.T[slice_ind,x_ind:,131:139], axis=(1)).T
ratio_roi = val_roi/val_adj

ratio_roi = gaussian_filter(ratio_roi, sigma=2)
# for i, ratio in enumerate(ratio_roi.T):
plt.plot(ratio_roi,'-', color='indianred',label='$\mu_{high}$')
plt.ylabel('Relative $\mu_{high}$')
plt.fill_betweenx([0.98,1.4], 18, 68, color='indianred', alpha=0.2)#,label='Metal Region')
plt.fill_betweenx([0.98,1.4], 93, 112, color='indianred', alpha=0.2)
plt.axhline(y=1.1, color='k', linestyle='-.',label='Threshold 1')
plt.legend(loc='lower left')
# plt.tight_layout()
# plt.figure(figsize=(5,10))
plt.subplot(221)
ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
plt.imshow(first_layer_bilat_proj.T[slice_ind][150:,50:],cmap='gray')
plt.title('(a) Rotated Projection image')
# plt.axvline(x=131-50, color='indianred', linestyle=':')
# plt.axvline(x=139-50, color='indianred', linestyle=':',label='Metal Region')
plt.fill_betweenx([0, first_layer_bilat_proj.T[slice_ind][150:,50:].shape[0]-2], 129-50, 141-50, color='indianred', alpha=0.2,label='Metal Region')
ratio[np.isinf(ratio)] = 0
ratio[first_layer_bilat_proj.T<6] = 0
ratio[ratio > 1.07] = 0
ratio[ratio < 1.01] = 0

plt.yticks([])
plt.legend()
plt.subplot(223)
plt.plot(np.nanmean(ratio, axis=(0,1))[50:])
plt.xlabel('Pixel (vertical direction)')
plt.ylabel('Mean ratio $\mu_{high}/\mu_{low}$')
plt.title('(c) Vertical Metal Selection')
# draw a red line horizontally at y = 0.7
plt.axhline(y=0.07, color='k', linestyle='-.',label='Threshold')
# draw horizontal line at x=131 and x=139
# plt.axvline(x=131, color='indianred', linestyle=':')
# plt.axvline(x=139, color='indianred', linestyle=':')#,label='ROI')
# # Fill between the metal boundaries
plt.fill_betweenx([0, 0.12], 129-50, 141-50, color='indianred', alpha=0.2)#,label='Metal Region')
plt.legend(loc='upper right')
plt.tight_layout()

# plt.savefig('metal_selection.png',dpi=300)

/tmp/ipykernel_20552/877753966.py:8: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/877753966.py:8: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/877753966.py:89: RuntimeWarning: divide by zero encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/877753966.py:89: RuntimeWarning: invalid value encountered in divide
  ratio = first_layer_bilat_proj.T[:]/secon_layer_bilat_proj.T[:]
/tmp/ipykernel_20552/877753966.py:115: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [97]:
plt.figure()
plt.plot(np.nansum(gt.T[129:142,:].astype(np.float32),axis=0))#cmap='turbo')

/tmp/ipykernel_3967/3301560325.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


In [11]:
yxall = np.nanmean(ratio, axis=(1))

In [12]:
plt.figure()
plt.plot(xall)

In [74]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def trace_replacement_spine(images0,images1,images4):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    images4 = images4.copy()
    mask_gts = []
    masks = []
    
    for image_high, image_low, image_low2 in zip(images2, images3, images4):
        ratio1 = image_high/(image_low2)

        ratio0 = image_high/(image_low)
        ratio0[image_high < 4] = 0
        # ratio1 = image_high/(image_low2)

        mask = np.zeros_like(image_high)
        # mask_gts.append(ratio1[240:,125:145 ] > 1.05)
        # p_98 = np.percentile(image_high, 97)
        # p_982 = np.percentile(ratio0, 95)
        # mask[240:290,130:140 ] += image_high[240:290,130:140 ] > p_98
        # mask[240:290,130:140 ] += ratio0[240:290,130:140 ] > p_982

        for row in range(240,mask.shape[0]):
            
            # p_98_row = np.percentile(image_high[row], 99.98)
            # p_98_row_ratio = np.percentile(ratio0[row], 99.98)
            # if np.max(image_high[row,131:139]) > p_98_row:#or np.max(image_high[row-2: row+2,131:139]) >7.0:
            #     mask[row,130:140] = 1
            # if np.max(ratio0[row,131:139]) > p_98_row_ratio:# or np.max(ratio0[row-2: row+2,131:139]) >1.03:
            #     mask[row,130:140] = 1

            if np.mean(image_high[row,131:139])/np.mean(image_high[row,110:130]) > 1.1:
                mask[row,130:140] = 1 
            if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.1:
                mask[row,130:140] = 1
        
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        # results.append(image_result)
        image_high[mask] = image_result[mask]
        results.append(image_high)
        masks.append(mask[240:,125:145 ])

        # mask = np.zeros_like(image_high)
        # mask_gts.append(ratio1[240:,125:145 ] > 1.05)
        mask_gts.append(ratio1[240:,125:145 ] > 1.05)
    
    # compute the dice score between the masks and the ground truth masks
    dice_scores = []
    # Compute the false positive and false negative rates
    false_positive = []
    false_negative = []
    for mask, mask_gt in zip(masks, mask_gts):
        dice_scores.append(2*np.sum(mask*mask_gt)/(np.sum(mask) + np.sum(mask_gt)))
        false_positive.append(np.sum(mask*(1-mask_gt))/np.sum(mask))
        false_negative.append(np.sum((1-mask)*mask_gt)/np.sum(mask_gt))
    print('Dice scores: ', np.mean(dice_scores))
    print('False positive: ', np.mean(false_positive))
    print('False negative: ', np.mean(false_negative))
    return results

images_LI = trace_replacement_spine(first_layer_bilat_proj.T[:],secon_layer_bilat_proj.T,first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_20552/4079708716.py:21: RuntimeWarning: divide by zero encountered in divide
  ratio1 = image_high/(image_low2)
/tmp/ipykernel_20552/4079708716.py:21: RuntimeWarning: invalid value encountered in divide
  ratio1 = image_high/(image_low2)
/tmp/ipykernel_20552/4079708716.py:23: RuntimeWarning: divide by zero encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_20552/4079708716.py:23: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_20552/4079708716.py:45: RuntimeWarning: invalid value encountered in scalar divide
  if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.1:
/tmp/ipykernel_20552/4079708716.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.1:


Dice scores:  0.6847458412643672
False positive:  0.4742348456807251
False negative:  0.010313902762075925


In [79]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def trace_replacement_spine(images0,images1,images4):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()
    images4 = images4.copy()
    mask_gts = []
    masks = []
    
    for image_high, image_low, image_low2 in zip(images2, images3, images4):
        ratio1 = image_high/(image_low2)

        ratio0 = image_high/(image_low)
        ratio0[image_high < 4] = 0

        mask = np.zeros_like(image_high)
        # mask[240:,125:145 ] = ratio0[240:,125:145 ] > 1.05
        # p_98 = np.percentile(image_high, 97)
        # p_982 = np.percentile(ratio0, 95)
        # mask[240:290,130:140 ] += image_high[240:290,130:140 ] > p_98
        # mask[240:290,130:140 ] += ratio0[240:290,130:140 ] > p_982

        for row in range(240,mask.shape[0]):
            
            # p_98_row = np.percentile(image_high[row], 99.98)
            # p_98_row_ratio = np.percentile(ratio0[row], 99.98)
            # if np.max(image_high[row,131:139]) > p_98_row:#or np.max(image_high[row-2: row+2,131:139]) >7.0:
            #     mask[row,130:140] = 1
            # if np.max(ratio0[row,131:139]) > p_98_row_ratio:# or np.max(ratio0[row-2: row+2,131:139]) >1.03:
            #     mask[row,130:140] = 1

            if np.mean(image_high[row,131:139])/np.mean(image_high[row,110:130]) > 1.1:
                mask[row,130:140] = 1 
            if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.03:
                mask[row,130:140] = 1 
            if np.max(image_high[row,131:139])/np.max(image_high[row,110:130]) > 1.2:
                mask[row,130:140] = 1
            if np.max(ratio0[row,131:139])/np.max(ratio0[row,110:130]) > 1.05:
                mask[row,130:140] = 1
            
            # if row == mask.shape[0]-1 or row == mask.shape[0]-2:
            #     mask[row,130:140] = 0
            #     image_high[row,130:140] = 0
            # if row == mask.shape[0]-3 or row == mask.shape[0]-4:
            #     mask[row,130:140] = 0
            #     image_high[row,130:140] = 0
            
        mask[mask == 2] = 1
        mask[mask == 3] = 1
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)
        # expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        # results.append(image_result)
        image_high[mask] = 0.7*image_low2[mask] + 0.3*image_result[mask]
        results.append(image_high)
        masks.append(mask[240:,125:145 ])
        

        # mask = np.zeros_like(image_high)
        mask_gts.append(ratio1[240:,125:145 ] > 1.05)
        # mask_gts.append(binary_dilation(binary_dilation(ratio1[240:,125:145 ] > 1.03)))
        # mask_gts = mask_gts)
    
    # compute the dice score between the masks and the ground truth masks
    dice_scores = []
    # Compute the false positive and false negative rates
    false_positive = []
    false_negative = []
    for mask, mask_gt in zip(masks, mask_gts):
        dice_scores.append(2*np.sum(mask*mask_gt)/(np.sum(mask) + np.sum(mask_gt)))
        false_positive.append(np.sum(mask*(1-mask_gt))/np.sum(mask))
        false_negative.append(np.sum((1-mask)*mask_gt)/np.sum(mask_gt))
    print('Dice scores: ', np.mean(dice_scores))
    print('False positive: ', np.mean(false_positive))
    print('False negative: ', np.mean(false_negative))
    return results

images_LI_tr = trace_replacement_spine(first_layer_bilat_proj.T[:],first_layer_bilat_proj.T,first_layer_bilat_proj_nm.T[:])

/tmp/ipykernel_20552/1358036806.py:21: RuntimeWarning: divide by zero encountered in divide
  ratio1 = image_high/(image_low2)
/tmp/ipykernel_20552/1358036806.py:21: RuntimeWarning: invalid value encountered in divide
  ratio1 = image_high/(image_low2)
/tmp/ipykernel_20552/1358036806.py:23: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_20552/1358036806.py:44: RuntimeWarning: invalid value encountered in scalar divide
  if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.03:
/tmp/ipykernel_20552/1358036806.py:48: RuntimeWarning: invalid value encountered in scalar divide
  if np.max(ratio0[row,131:139])/np.max(ratio0[row,110:130]) > 1.05:
/tmp/ipykernel_20552/1358036806.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  if np.mean(ratio0[row,131:139])/np.mean(ratio0[row,110:130]) > 1.03:
/tmp/ipykernel_20552/1358036806.py:48: RuntimeWarning: divide by zero encountered in scalar divide
  if np.max(ratio

Dice scores:  0.6846669879983289
False positive:  0.474325159169994
False negative:  0.010313902762075925


In [52]:
geo = tigre.geometry_default()

In [80]:
# plt.figure()
# plt.plot(secon_layer_bilat_header[0][-1][2][1,:], secon_layer_bilat_header[0][-1][2][0,:], 'o')
# plt.plot(first_layer_bilat_header[0][-1][1][1,:], first_layer_bilat_header[0][-1][1][0,:], 'o')

source_pos_x = secon_layer_bilat_header[0][-1][2][1,:]
source_pos_y = secon_layer_bilat_header[0][-1][2][0,:]
detector_pos_x = first_layer_bilat_header[0][-1][1][1,:]
detector_pos_y = first_layer_bilat_header[0][-1][1][0,:]

# calculate the angles from the source to the detector
angles_real = np.arctan2(detector_pos_x - source_pos_x, detector_pos_y - source_pos_y)

# calculate the distance from the source to the detector
distance = np.sqrt((detector_pos_x - source_pos_x)**2 + (detector_pos_y - source_pos_y)**2)


In [81]:
import h5py
import numpy as np

class Geo:
    pass

def assign_values_to_geo(filename):
    geo = Geo()
    with h5py.File(filename, 'r') as f:
        def assign_value(name, obj):
            if isinstance(obj, h5py.Dataset):
                # Remove 'geo/' from the name and replace any remaining '/' with '_'
                attr_name = name.replace('geo/', '').replace('/', '_')
                setattr(geo, attr_name, np.array(obj[0][()]))
        f.visititems(assign_value)
    return geo

# Use the function
geo2 = assign_values_to_geo('/home/jericho/Downloads/firefox-downloads/geo.mat')

# Now you can access the values like this:
print(geo2.DSD)
print(geo2.offDetector)
# print_hdf5_file_structure()

[1500.]
[160.   0.]


In [82]:
geo.DSD = 1527.3
geo.DSO = 1000
# geo.mode = geo2.mode
geo.nDetector = np.array([266, 354])
geo.dDetector = np.array([1.12,1.12])
geo.sDetector = geo.nDetector*geo.dDetector
geo.offDetector = np.array([0,-160])# geo2.offDetector
geo.nVoxel = np.array([120,512,512])
geo.dVoxel = np.array([1.2,1,1])
geo.sVoxel = geo.nVoxel*geo.dVoxel
geo.offOrigin = geo2.offOrigin

In [83]:
angles = angles_real 
# Shift the angles by 110 degrees 
angles = angles + np.deg2rad(-87)
# Make sure that the angles are between 0 and 2*pi
angles = np.mod(angles, 2*np.pi)


In [84]:
img_first = tigre.algorithms.fdk(first_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
img_second = tigre.algorithms.fdk(secon_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
img_LI = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1]),geo,angles,dowang=True)
img_gt = tigre.algorithms.fdk(first_layer_bilat_proj_nm.transpose([2,0,1]),geo,angles,dowang=True)
# img_combined = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)
FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
D

In [85]:
img_tr = tigre.algorithms.fdk(np.array(images_LI_tr).transpose([0,2,1]),geo,angles,dowang=True)
# img_wtr = tigre.algorithms.fdk(np.array(images_LI_wtr).transpose([0,2,1]),geo,angles,dowang=True)
# img_LI_second = tigre.algorithms.fdk(np.array(images_LI_second).transpose([0,2,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [86]:
def mask_center(image, radius):
    # Get the center of the image
    center = np.array(image.shape) / 2

    # Create an array of the same shape as the image, filled with the squared distance from the center
    y, x = np.ogrid[:image.shape[0], :image.shape[1]]
    dist_from_center = (x - center[1])**2 + (y - center[0])**2

    # Create a mask that is True for all pixels within the specified radius of the center
    mask = dist_from_center <= radius**2

    # Use the mask to set the corresponding pixels in the original image to 0
    image[~mask] = 0

    return image

# # crop the image from 512 to 416
# img = img[:,48:464, 48:464]
# # Test the function
# image = np.ones((100, 100))
radius = 416//2
# for im in img:
#     im = mask_center(im, radius)

img_LI0 = img_LI[:,48:464, 48:464].copy()
for im in img_LI0:
    im = mask_center(im, radius)

# img[img < 0.01] = 0
img_LI0[img_LI0 < 0.01] = 0

# do the same for the first and second layers

img_first0 = img_first[:,48:464, 48:464].copy()
for im in img_first0:
    im = mask_center(im, radius)

img_second0 = img_second[:,48:464, 48:464].copy()
for im in img_second0:
    im = mask_center(im, radius)

img_gt0 = img_gt[:,48:464, 48:464].copy()
for im in img_gt0:
    im = mask_center(im, radius)

img_tr0 = img_tr[:,48:464, 48:464].copy()
for im in img_tr0:
    im = mask_center(im, radius)

# img_one0 = img_one[:,48:464, 48:464].copy()
# for im in img_one0:
#     im = mask_center(im, radius)

# coeff = 0

# img_first0 -= coeff*img_one0
# img_second0 -= coeff*img_one0
# img_LI0 -= coeff*img_one0
# img_gt0 -= coeff*img_one0
# img_tr0 -= coeff*img_one0

img_first0[img_first0 < 0.01] = 0
img_first0[img_first0 < 0.01] = 0
img_second0[img_second0< 0.01] = 0
img_gt0[img_gt0 < 0.01] = 0
img_tr0[img_tr0 < 0.01] = 0


# convert each of the images to HU using the water and air values
water = 0.0182
air = 0
img_LI0 = (img_LI0 - air)/(water - air)*1000 - 1000
img_first0 = (img_first0 - air)/(water - air)*1000 - 1000
img_second0 = (img_second0 - air)/(water - air)*1000    - 1000
img_gt0 = (img_gt0 - air)/(water - air)*1000 - 1000
img_tr0 = (img_tr0 - air)/(water - air)*1000 - 1000
# img_wtr0 = (img_wtr0 - air)/(water - air)*1000 - 1000
# img_LI_second0 = (img_LI_second0 - air)/(water - air)*1000 - 1000

im_num = 61

plt.figure(figsize=(6,6))
plt.subplot(2,2,3)
plt.imshow(img_first0[im_num],vmin=-400,vmax=800,cmap='gray')
plt.title('(c) First Layer')
plt.axis('off')
plt.subplot(2,2,4)
plt.imshow(img_second0[im_num],vmin=-400,vmax=800,cmap='gray')
plt.title('(d) Second Layer')
plt.axis('off')
# plt.subplot(3,2,3)
# plt.imshow(img_LI0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Biharmonic Inpainting')
# plt.axis('off')
plt.subplot(221)
plt.imshow(img_gt0[im_num],vmin=-400,vmax=800,cmap='gray')
plt.title('(a) Ground Truth')
plt.axhline(y=245, color='indianred', linestyle=':',label='Plotted Region')
plt.axis('off')
plt.legend()
plt.subplot(222)
plt.imshow(img_tr0[im_num],vmin=-400,vmax=800,cmap='gray')
plt.title('(b) RMR')
plt.axis('off')
# plt.subplot(3,2,6)
# plt.imshow(img_LI_second0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Biharmonic Inpainting Second Layer')
# plt.axis('off')

plt.tight_layout()
# plt.savefig('RMR_vs_GT.png',dpi=300)

In [87]:
# Extract the line plotted region from each image
line_gt = img_gt0[im_num, 245, :]
line_first = img_first0[im_num, 245, :]
line_second = img_second0[im_num, 245, :]
line_tr = img_tr0[im_num, 245, :]

# Create a new figure for the line plots
plt.figure(figsize=(6, 4))

# Plot the line plotted region for each image
plt.plot(line_gt, label='Ground Truth')
plt.plot(line_first, label='First Layer')
plt.plot(line_second, label='Second Layer')
plt.plot(line_tr, label='RMR')

# Add a legend
plt.legend()

# Add labels and title
plt.xlabel('Pixel Position')
plt.ylabel('Pixel Value')
plt.title('Metal Artifact Region Comparison')

# Show the plot
plt.show()
# plt.savefig('RMR_vs_GT_line.png',dpi=300)

# print the rmses between the gt and the other lines 
rmse_first = np.sqrt(np.mean((line_gt - line_first)**2))
rmse_second = np.sqrt(np.mean((line_gt - line_second)**2))
rmse_tr = np.sqrt(np.mean((line_gt - line_tr)**2))

print(f'RMSE between the GT and the first layer: {rmse_first}')
print(f'RMSE between the GT and the second layer: {rmse_second}')
print(f'RMSE between the GT and the RMR: {rmse_tr}')

# do the same thing but averoaging over 5 lines axially to remove noise
line_gt = np.mean(img_gt0[im_num, 240:250, :], axis=0)
line_first = np.mean(img_first0[im_num, 240:250, :], axis=0)
line_second = np.mean(img_second0[im_num, 240:250, :], axis=0)
line_tr = np.mean(img_tr0[im_num, 240:250, :], axis=0) 

# Just print the rmse 
rmse_first = np.sqrt(np.mean((line_gt - line_first)**2))
rmse_second = np.sqrt(np.mean((line_gt - line_second)**2))
rmse_tr = np.sqrt(np.mean((line_gt - line_tr)**2))

print(f'RMSE between the GT and the first layer: {rmse_first}')
print(f'RMSE between the GT and the second layer: {rmse_second}')
print(f'RMSE between the GT and the RMR: {rmse_tr}')

RMSE between the GT and the first layer: 614.6217041015625
RMSE between the GT and the second layer: 502.48333740234375
RMSE between the GT and the RMR: 46.94968032836914
RMSE between the GT and the first layer: 427.89349365234375
RMSE between the GT and the second layer: 359.7525634765625
RMSE between the GT and the RMR: 25.30536460876465


In [188]:

plt.figure(figsize=(10,10))
# plt.subplot(1,1,1)
plt.imshow(img_first0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('First Layer')
plt.axis('off')
plt.savefig('spine_first.png',dpi=300)
# plt.subplot(3,2,2)
# plt.imshow(img_second0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Second Layer')
# plt.axis('off')
# plt.subplot(3,2,3)
# plt.imshow(img_LI0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Biharmonic Inpainting')
# plt.axis('off')
# plt.subplot(3,2,4)
# plt.imshow(img_gt0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Ground Truth')
# plt.axis('off')

plt.figure(figsize=(10,10))
plt.imshow(img_tr0[62],vmin=-400,vmax=400,cmap='gray')
# plt.title('Trace Replacement')
plt.axis('off')
# plt.subplot(3,2,6)
plt.savefig('spine_replacement.png',dpi=300)

In [66]:
# plt.figure()

# for ii, alpha in enumerate([0.1,0.5,1.0,2.0]):
#     img_sub = tigre.algorithms.fdk((first_layer_bilat_proj - alpha*secon_layer_bilat_proj).transpose([2,0,1]),geo,angles,dowang=True)
#     img_sub0 = img_sub[:,48:464, 48:464].copy()
#     # for im in img_sub0:
#     #     im = mask_center(im, radius)
#     # img_sub0[img_sub0 < 0.01] = 0
#     # img_sub0 = (img_sub0 - air)/(water - air)*1000 - 1000
#     plt.subplot(2,2,ii+1)
#     plt.imshow(img_sub0[62],cmap='gray')
#     plt.title('Subtraction with alpha = ' + str(alpha))
#     plt.axis('off')

img_sub = tigre.algorithms.fdk((first_layer_bilat_proj - 2*secon_layer_bilat_proj).transpose([2,0,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [68]:
images = []

for coeff in np.linspace(0,100,100):
    images.append(img_first[62] - img_sub[62]*(coeff/5))

plt.figure()
cc = pli.hyperslicer(images)#,vmin=0,vmax=0.05)

In [62]:
plt.figure()
plt.imshow(10*img_sub[62]-img_first[62],cmap='gray')

In [18]:
# load an h5py  file
import h5py
import numpy as np

def print_hdf5_file_structure(file_name):
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print("    %s: %s" % (key, val))

    with h5py.File(file_name, 'r') as f:
        f.visititems(print_attrs)

fname = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

print_hdf5_file_structure(fname)

LI_CT
LI_sinogram
ma_CT
ma_sinogram
metal_trace


In [28]:
def test_image(data_path, imag_idx, mask_idx):
    txtdir = os.path.join(data_path, 'test_640geo_dir.txt')
    mat_files = open(txtdir, 'r').readlines()
    gt_dir = mat_files[imag_idx]
    file_dir = gt_dir[:-6]
    data_file = file_dir + str(mask_idx) + '.h5'
    abs_dir = os.path.join(data_path, 'test_640geo/', data_file)
    gt_absdir = os.path.join(data_path, 'test_640geo/', gt_dir[:-1])
    gt_file = h5py.File(gt_absdir, 'r')
    Xgt = gt_file['image'][()]
    gt_file.close()
    file = h5py.File(abs_dir, 'r')
    Xma= file['ma_CT'][()]
    XLI =file['LI_CT'][()]
    file.close()

    return Xgt, Xma, XLI
    # M512 = test_mask[:,:,mask_idx]
data_path = '/home/jericho/Software/dl_mar/OSCNet/data/test'
Xgt, Xma, XLI = test_image(data_path, 0, 0)


In [30]:
plt.figure()
pli.hyperslicer([Xgt,Xma,XLI])

AttributeError: 'Controls' object has no attribute 'vbox'

In [26]:
plt.figure()
plt.hist(Xma.flatten(), histtype='step',bins=100, alpha=0.5, label='ma_ct',log=True)
plt.hist(XLI.flatten(), histtype='step',bins=100, alpha=0.5, label='li_ct',log=True)
plt.hist(img[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_ma_ct',log=True)
plt.hist(img_LI[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_li_ct',log=True)
plt.legend()

In [27]:
# Open 0.h5 replace Xma with the img[55] and save as h5py file 1.h5
import h5py

fname2 = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

# Load the file
import h5py
import numpy as np

# Open the file in 'r+' mode
with h5py.File(fname2, 'r+') as f:
    # Load the images 
    f['ma_CT'][()] = img[62]
    f['LI_CT'][()] = img_LI[62]
    # The changes are automatically saved when the file is closed

In [94]:
%matplotlib qt

#%% DEMO 14:  Playing with offsets
#
#
# In this demo we show how to change offsets to either the iamge or the
# detector, and the flexibility of it.
#
#
# --------------------------------------------------------------------------
# --------------------------------------------------------------------------
# This file is part of the TIGRE Toolbox
#
# Copyright (c) 2015, University of Bath and
#                     CERN-European Organization for Nuclear Research
#                     All rights reserved.
#
# License:            Open Source under BSD.
#                     See the full license at
#                     https://github.com/CERN/TIGRE/blob/master/LICENSE
#
# Contact:            tigre.toolbox@gmail.com
# Codes:              https://github.com/CERN/TIGRE/
# Coded by:           Ander Biguri
# --------------------------------------------------------------------------
## Initialize
#%%Initialize
import tigre
import numpy as np
from tigre.utilities import sample_loader
import matplotlib.pyplot as plt
import tigre.algorithms as algs

#%% Geometry
geo = tigre.geometry_default(high_resolution=False)


# Offsets
## ####################################################################################
# Lets try simnple offset: The detector gets completelly displaced
geo.offOrigin = np.array([0, 0, 0])  # Offset of image from origin   (mm)
geo.offDetector = np.array([200, 200])  # Offset of Detector            (mm)
## ####################################################################################
# Auxiliary
geo.accuracy = 0.5
# Accuracy of FWD proj          (vx/sample)

## Load data and generate projections
# see previous demo for explanation
angles = np.linspace(0, 2 * np.pi, 100)

# Load thorax phatom data
head = sample_loader.load_head_phantom(geo.nVoxel)
projections = tigre.Ax(head, geo, angles)


#%% lets see it
# tigre.plotproj(projections, angles)

## we will skip reconstruction of this tests because the image is outside the detector


## #####################################################################

#%% Second test: lets test variying offsets:
geo.nDetector = np.array([80,120])  # Number of detector pixels
geo.dDetector = np.array([4, 4])  # Detector element size (mm)
geo.sDetector = geo.nDetector * geo.dDetector  # Detector size (mm)
geo.offDetector = np.array(
    [0,-160]
)  # Offset of Detector            (mm)
projections2 = tigre.Ax(head, geo, angles)
## lets see it
# tigre.plotproj(projections2, angles)
## reconstruction
geo.offDetector = np.array(
    [0,-160]
)  

res = algs.fdk(projections2, geo, angles, dowang=True)


# tigre.plotimg(res, dim="z")

#%% Third test: lets vary everything
# plt.show()

FDK: applying detector offset weights
-80 200 0 -80.0
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1536.0 mm
Distance from source to origin (DSO)= 1000.0 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 80 200]
Size of each pixel (dDetector) = [4 4] mm
Total size of the detector (sDetector) = [320 800] mm
-----
Image parameters
Number of voxels (nVoxel) = [64 64 64]
Total size of the image (sVoxel) = [256 256 256] mm
Size of each voxel (dVoxel) = [4. 4. 4.] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad
(100, 80, 200)


In [86]:
# load a png file and plot it using imshow
png_file = '/home/jericho/Software/dl_mar/OSCNet/save_results/osc/1.png'
import mpl_interactions.ipyplot as iplt
img = plt.imread(png_file)
plt.figure()
cc = iplt.imshow(img.mean(axis=2),vmin_vmax=(0,1))

In [96]:
plt.figure()
pli.hyperslicer(res)

AttributeError: 'Controls' object has no attribute 'vbox'

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import tigre

In [8]:
import mpl_interactions as pli


plt.figure()
# pli.hyperslicer(first_layer_bilat_proj.transpose([2,0,1]))
pli.hyperslicer(projections2)

AttributeError: 'Controls' object has no attribute 'vbox'